In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://cloud.r-project.org/b

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-04 21:58:33--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.81MB/s    in 0.2s    

2020-11-04 21:58:33 (4.81 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [18]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://laligadb.s3.amazonaws.com/diabetes.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("diabetes.csv"), header =True, inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+-----+------+---+----+---------+-------+------+--------------+---------+----------+------+---+---+---+---+----+------+--------+------------+------+---------+---------+----+--------+------+------+------------+---+-----+----+--------+----+---+------+------+------+----+-------+-----+-----+----+-----------+---------+------+------+---------+--------+----+-----+---+----+----+---------+--------------------+---+----+-------------+---------+---------+--------+
| seqn|gender|age|race|education|marital|income|household_size|insurance|gen_health|asthma|chf|cad| mi|cva|copd|cancer|diabetes|hypertension|smoker|weight_kg|height_cm| bmi|waist_cm|sys_bp|dia_bp|alb_cr_ratio|alb|  alt| ast|alk_phos| bun| ca|   cpk|t_chol|bicarb|  cr|glucose| iron|  ldh|phos|t_bilirubin|t_protein|u_acid|sodium|potassium|chloride|glob|trigs|wbc| hgb| hct|platelets|          s_cotinine|a1c| hdl|grip_strength|bmi_group|age_group|bp_group|
+-----+------+---+----+---------+-------+------+--------------+---------+----------+--

In [19]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cgd9exyjeqgb.us-east-2.rds.amazonaws.com:5432/meta_b_database"
config = {"user":"root", 
          "password": "12345678", 
          "driver":"org.postgresql.Driver"}

In [20]:
df.write.jdbc(url=jdbc_url, table='diabetes_info', mode=mode, properties=config)